# Лабораторная работа 1. Нейронные сети.

Результатом лабораторной работы является отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете также должен быть код, однако чем меньше кода, тем лучше всем: нам — меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

Мы уверены, что выполнение лабораторных работ занимает значительное время, поэтому не рекомендуем оставлять их на последний вечер перед сдачей.


**ВНИМАНИЕ:** В рамках данной лабораторной работы допускается использование PyTorch вместо Keras

# Часть 1. Свёрточные сети

Здесь вам предстоит построить и обучить свою первую свёрточную сеть для классификации изображений на данных CIFAR10. 

## Данные

CIFAR10
* 60000 RGB изображений размером 32x32x3
* 10 классов: самолёты, собаки, рыбы и т.п.

<img src="https://www.samyzaf.com/ML/cifar10/cifar1.jpg" style="width:60%">

Загрузите данные, разделите их на обучающую и тестовую выборки. Размер тестовой выборки должен быть 10^4.

In [ ]:
from torch import nn
import torch

In [ ]:
import numpy as np
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10**4, random_state=42)

class_names = np.array(['airplane','automobile ','bird ','cat ','deer ','dog ','frog ','horse ','ship ','truck'])

print(X_train.shape, y_train.shape)

Датасет также доступен по ссылке [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html). В PyTorch нужно использовать [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html) и разбить его на обучающую, валидационную и тестовую выборки.

**Замечание:** По умолчанию данные в PyTorch разбиты на обучающую и тестовую выборки. Для того что бы разбить обучающую выборку на обучающую и валидационную, можно воспользоваться [torch.utils.data.sampler.SubsetRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.SubsetRandomSampler).

Прежде, чем приступать к основной работе, стоит убедиться, что загруженно именно то, что требовалось:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=[12,10])
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.xlabel(class_names[y_train[i, 0]])
    plt.imshow(X_train[i])

## Подготовка данных

Сейчас каждый пиксель изображения закодирован тройкой чисел (RGB) __от 0 до 255__. Однако лучше себя показывает подход, где значения входов нейросети распределены недалеко от 0.

Давайте приведём все данные в диапазон __`[0, 1]`__ — просто разделим на соответствующий коэффициент:

In [ ]:
X_train = (X_train / 255).astype('float32')
X_val =  (X_val / 255).astype('float32')
X_test = (X_test / 255).astype('float32')

Исполните код ниже для проверки, что все выполнено корректно.

In [ ]:
assert np.shape(X_train) == (40000, 32, 32, 3), "data shape should not change"
assert 0.9 <= max(map(np.max, (X_train, X_val, X_test))) <= 1.05
assert 0.0 <= min(map(np.min, (X_train, X_val, X_test))) <= 0.1
assert len(np.unique(X_test / 255.)) > 10, "make sure you casted data to float type"

## Архитектура сети

Для начала реализуйте простую нейросеть:
1. принимает на вход картинки размера 32 x 32 x 3;
2. вытягивает их в вектор (`keras.layers.Flatten`, `torch.nn.Flatten`);
3. пропускает через 1 или 2 полносвязных слоя;
4. выходной слой отдает вероятности принадлежности к каждому из 10 классов.

Создайте полносвязную сеть:

In [ ]:
simple_dense_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(32*32*3, 2048),
    nn.ReLU(),
    nn.Linear(2048, 1024),
    nn.ReLU(), 
    nn.Linear(1024, 10), 
    nn.Softmax(dim=1)
    )

In [ ]:
dummy_pred = simple_dense_model(torch.from_numpy(X_train[:20])).detach().numpy()
assert dummy_pred.shape == (20, 10)
assert np.allclose(dummy_pred.sum(-1), 1)

print("Успех!")

### Создание копии модели:

## Обучение сети

**Задание 1.1 (1.5 балла).** Будем минимизировать многоклассовую кроссэнтропию с помощью __sgd__. Вам нужно получить сеть, которая достигнет __не менее 45%__ __accuracy__ на тестовых данных.

__Важно:__ поскольку в y_train лежат номера классов, Керасу нужно либо указать sparse функции потерь и метрики оценки качества классификации (`sparse_categorical_crossentropy` и `sparse_categorical_accuracy`), либо конвертировать метки в one-hot формат. PyTorch, напротив, умеет работать с меткаим классов (`torch.nn.CrossEntropyLoss`)

### Полезные советы (keras)
* `model.compile` позволяет указать, какие метрики вы хотите вычислять.
* В `model.fit` можно передать валидационную выборку (`validation_data=[X_val, y_val]`), для отслеживания прогресса на ней. Также рекомендуем сохранять результаты в [tensorboard](https://keras.io/callbacks/#tensorboard).
* По умолчанию сеть учится 1 эпоху. Совсем не факт, что вам этого хватит. Число эпох можно настроить в методе `fit` (`epochs`).
* Ещё у Кераса есть много [полезных callback-ов](https://keras.io/callbacks/), которые можно попробовать. Например, автоматическая остановка или подбор скорости обучения.

### PyTorch
В PyTorch есть модуль [tensorboard](https://pytorch.org/docs/stable/tensorboard.html),  который по сути использует готоывй (установленный вами) TensorBoard, так что в вопросе визуализации PyTorch несколько проигрывает Tensorflow, но, тем не менее, благодаря данному модулю все возможности Tensorboard досутпны и в PyTorch

In [ ]:
import torchvision
from torchvision.transforms import ToTensor


dataset = torchvision.datasets.CIFAR10('cifar10/train/', download=True, transform=ToTensor(), train=True)
test_data = torchvision.datasets.CIFAR10('cifar10/test', download=True, transform=ToTensor(), train=False)

train_data, val_data = train_test_split(dataset, test_size=5000, random_state=42)

train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)

val_loader = torch.utils.data.DataLoader(val_data,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=2)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=2)

In [ ]:
X_test = torch.Tensor(10000, 3, 32, 32)
y_test = []
for i, data in enumerate(test_data):
    image, label = data
    X_test[i,:,:,:] = image
    y_test.append(label)
y_test = torch.Tensor(y_test)

In [ ]:
from pytorch_lightning.core.lightning import LightningModule
from sklearn.metrics import accuracy_score
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from copy import deepcopy

In [ ]:
class CifarModel(LightningModule):
    def __init__(self, model, get_optimizer = None):
        super().__init__()
        self.model = model
        self.loss = nn.CrossEntropyLoss()
        if get_optimizer:
            self.optimizer = get_optimizer(self.parameters())
        else:
            self.optimizer = SGD(self.parameters(), lr=1e-2, momentum=0.9)


    def forward(self, x):
        return self.model.forward(x)
      
    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(y.cpu(), preds.cpu())
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
      
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(y.cpu(), preds.cpu())
        
        self.log('val_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy_score(y.cpu(), preds.cpu())
        
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def predict_step(self, batch, batch_idx: int, dataloader_idx: int = None):
        x,y = batch
        return self(x).argmax(axis=1)

In [ ]:
logger = TensorBoardLogger("logs/task1")
dense_model = CifarModel(deepcopy(simple_dense_model))
trainer = Trainer(logger=logger, gpus=1, max_epochs=25)
trainer.fit(dense_model, train_loader, val_loader)

А теперь можно проверить качество вашей сети, выполнив код ниже:

In [ ]:
y_pred = torch.cat(trainer.predict(dense_model, test_loader, return_predictions=True), dim=0).cpu()
test_acc = accuracy_score(y_test, y_pred)
print("\n Test_acc =", test_acc)
assert test_acc > 0.45, "Not good enough. Back to the drawing board :)"
print(" Not bad!")

**Примечание:**

Ячейка ниже запускает Tensorboard на кагле.

In [ ]:
# From Github Gist: https://gist.github.com/hantoine/4e7c5bc6748861968e61e60bab89e9b0
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
from subprocess import Popen
from os import chmod
from os.path import isfile
import json
import time
import psutil

def launch_tensorboard():
    tb_process, ngrok_process = None, None
    
    # Launch TensorBoard
    if not is_process_running('tensorboard'):
        tb_command = 'tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006'
        tb_process = run_cmd_async_unsafe(tb_command)
    
    # Install ngrok
    if not isfile('./ngrok'):
        ngrok_url = 'https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip'
        download_and_unzip(ngrok_url)
        chmod('./ngrok', 0o755)

    # Create ngrok tunnel and print its public URL
    if not is_process_running('ngrok'):
        ngrok_process = run_cmd_async_unsafe('./ngrok http 6006')
        time.sleep(1) # Waiting for ngrok to start the tunnel
    ngrok_api_res = urlopen('http://127.0.0.1:4040/api/tunnels', timeout=10)
    ngrok_api_res = json.load(ngrok_api_res)
    assert len(ngrok_api_res['tunnels']) > 0, 'ngrok tunnel not found'
    tb_public_url = ngrok_api_res['tunnels'][0]['public_url']
    print(f'TensorBoard URL: {tb_public_url}')

    return tb_process, ngrok_process


def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)


def run_cmd_async_unsafe(cmd):
    return Popen(cmd, shell=True)


def is_process_running(process_name):
    running_process_names = (proc.name() for proc in psutil.process_iter())
    return process_name in running_process_names


tb_process, ngrok_process = launch_tensorboard()

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs

## Карманная сверточная сеть

**Задание 1.2 (1.5 балла).** Реализуйте небольшую свёрточную сеть. Совсем небольшую:
1. Входной слой
2. Свёртка 3x3 с 10 фильтрами
3. Нелинейность на ваш вкус
4. Max-pooling 2x2
5. Вытягиваем оставшееся в вектор (Flatten)
6. Полносвязный слой на 100 нейронов
7. Нелинейность на ваш вкус
8. Выходной полносвязный слой с softmax

Обучите её так же, как и предыдущую сеть. Если всё хорошо, у вас получится accuracy не меньше __50%__.

In [ ]:
simple_convolutional_model = nn.Sequential(
    nn.Conv2d(3, 10, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(2250, 100),
    nn.ReLU(), 
    nn.Linear(100, 10), 
    nn.Softmax(dim=1)
    )

In [ ]:
logger = TensorBoardLogger("logs/task2")
conv_model = CifarModel(deepcopy(simple_convolutional_model))
trainer = Trainer(logger=logger, gpus=1, max_epochs=20)
trainer.fit(conv_model, train_loader, val_loader)

Давайте посмотрим, смогла ли карманная сверточная сеть побить заданный порог по качеству:

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = conv_model(X_test).argmax(axis=1).cpu()
test_acc = accuracy_score(y_test, y_pred)
print("\n Test_acc =", test_acc)
assert test_acc > 0.50, "Not good enough. Back to the drawing board :)"
print(" Not bad!")

## Учимся учить

А теперь научимся сравнивать кривые обучения моделей — зависимости значения accuracy от количества итераций. 

Вам потребуется реализовать _экспериментальный стенд_ — вспомогательный код, в который вы сможете подать несколько архитектур и методов обучения, чтобы он их обучил и вывел графики кривых обучения. Это можно сделать с помощью `keras.callbacks` — `TensorBoard` или `History`.

Будьте морально готовы, что на обучение уйдёт _много времени_. Даже если вы ограничитесь 10 эпохами. Пока идёт обучение, вы можете переключиться на другие задания или заняться чем-нибудь приятным: поспать, например.

In [ ]:
def train_model_experiment(model, task_name, train_loader, val_loader, test_loader, early_stopping = None, max_epoch=25):
    logger = TensorBoardLogger("logs/" + task_name)
    if early_stopping:
        trainer = Trainer(logger=logger, gpus=1, max_epochs=max_epoch, callbacks=[early_stopping])
    else:
        trainer = Trainer(logger=logger, gpus=1, max_epochs=max_epoch)
    trainer.fit(model, train_loader, val_loader)
    
    y_pred = torch.cat(trainer.predict(model, test_loader, return_predictions=True), dim=0).cpu()
    test_acc = accuracy_score(y_test, y_pred)
    return test_acc

**Задание 1.3 (1 балл).** Попробуйте использовать различные методы оптимизации (sgd, momentum, adam) с параметрами по умолчанию. Какой из методов работает лучше?

In [ ]:
model_sgd = CifarModel(deepcopy(simple_convolutional_model), lambda model_params: SGD(model_params, lr=2e-3))
sgd_acc = train_model_experiment(model_sgd, 'task3_sgd', train_loader, val_loader, test_loader)
sgd_acc

In [ ]:
model_momentum = CifarModel(deepcopy(simple_convolutional_model), lambda model_params: SGD(model_params, momentum=0.9, lr=2e-3))
momentum_acc = train_model_experiment(model_momentum, 'task3_momentum', train_loader, val_loader, test_loader)
momentum_acc

In [ ]:
model_adam = CifarModel(deepcopy(simple_convolutional_model), lambda model_params: Adam(model_params, lr=2e-3))
adam_acc = train_model_experiment(model_adam, 'task3_adam', train_loader, val_loader, test_loader)
adam_acc

### График **accuracy** на валидационной выборке:

* Коричный - SGD.
* Голубой - Momentum.
* Розовый - Adam.


<img src="https://drive.google.com/uc?export=view&id=1Se1Rll79TvXLxur55iNsz1E7iA98Pihl" width="600">

**Задание 1.4 (0.5 балла).** Добавьте нормализацию по батчу (`BatchNormalization`) между свёрткой и активацией. Попробуйте использовать несколько нормализаций — в свёрточных и полносвязных слоях.

In [ ]:
batch_norm_in_conv_model = nn.Sequential(
    nn.Conv2d(3, 10, 3),
    nn.BatchNorm2d(10),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(2250, 100),
    nn.ReLU(), 
    nn.Linear(100, 10), 
    nn.Softmax(dim=1)
    )
model_bn_conv = CifarModel(deepcopy(batch_norm_in_conv_model), lambda model_params: Adam(model_params, lr=2e-3))
bn_conv_acc = train_model_experiment(model_bn_conv, 'task4_batch_norm_conv', train_loader, val_loader, test_loader)
bn_conv_acc

In [ ]:
batch_norm_in_dense_model = nn.Sequential(
    nn.Conv2d(3, 10, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(2250, 100),
    nn.BatchNorm1d(100),
    nn.ReLU(), 
    nn.Linear(100, 10), 
    nn.Softmax(dim=1)
    )
model_bn_dense = CifarModel(deepcopy(batch_norm_in_conv_model), lambda model_params: Adam(model_params, lr=2e-3))
bn_dense_acc = train_model_experiment(model_bn_dense, 'task4_batch_norm_dense', train_loader, val_loader, test_loader)
bn_dense_acc

In [ ]:
batch_norm_model = nn.Sequential(
    nn.Conv2d(3, 10, 3),
    nn.BatchNorm2d(10),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(2250, 100),
    nn.BatchNorm1d(100),
    nn.ReLU(), 
    nn.Linear(100, 10), 
    nn.Softmax(dim=1)
    )

In [ ]:
model_bn = CifarModel(deepcopy(batch_norm_model), lambda model_params: Adam(model_params, lr=2e-3))
bn_acc = train_model_experiment(model_bn, 'task4_batch_norm', train_loader, val_loader, test_loader)
bn_acc

### График **accuracy** на валидационной выборке:

* Коричный - Батч-норм на полносвязном слое.
* Синий - Батч-норм после свертки.
* Зеленый - Батч-норм после свертки и на полносвязном слое.

<img src="https://drive.google.com/uc?export=view&id=14-_V0fd8qFigseAUubFw82BA-GhmsTWA" width="600">

**Задание 1.5 (0.5 балла).** Посмотрите на batch_size (параметр model.fit) - при большем батче модель будет быстрее проходить эпохи, но с совсем огромным батчем вам потребуется больше эпох для сходимости (т.к. сеть делает меньше шагов за одну эпоху).
Найдите такое значение, при котором модель быстрее достигает точности 55%.

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

batch_sizes_list = [4, 8, 16, 32, 64, 128]
early_stopping = EarlyStopping('val_acc', stopping_threshold=0.55, mode='max')
for batch_size in batch_sizes_list:
    train_loader_batch_task = torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=2)

    val_loader_batch_task = torch.utils.data.DataLoader(val_data,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=2)
    model = CifarModel(deepcopy(batch_norm_model), lambda model_params: Adam(model_params, lr=2e-3))
    acc = train_model_experiment(model, 'task5_batch_size' + str(batch_size), train_loader_batch_task, val_loader_batch_task, test_loader, early_stopping)
    print('Batch size: ' + str(batch_size))
    print('Accuracy: ' + str(acc))

### Вывод:
Модель с размером батча=4 получила требуемый **accuracy** за 9 эпох. Модель с размером батча 8 сделала это за 2 эпохи. Модели с размерами 16,32,64,128 получили требуемое качество за 1 эпоху. При этом, лучшее accuracy на тестовой выборке принадлежит модели с размером батча = 32.

**Задание 1.6 (0.5 балла).** Попробуйте найти такую комбинацию метода обучения и нормализации, при которой сеть имеет наилучшую кривую обучения. Поясните, что вы понимаете под "наилучшей" кривой обучения.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data,
                                          batch_size=32,
                                          shuffle=False,
                                          num_workers=2)
model = CifarModel(deepcopy(batch_norm_model), lambda model_params: Adam(model_params, lr=2e-3))
acc = train_model_experiment(model, 'task6_best_comb', train_loader, val_loader, test_loader, max_epoch=20)
acc

### График **accuracy** на валидационной выборке:

<img src="https://drive.google.com/uc?export=view&id=1a_T0b0ri_9lF77eerHLzTIXqE9aXygLo" width="600">

Напишите ваши выводы по проделанным экспериментам: что заработало, что — не очень. Постройте графики кривых обучения разных архитектур.

## Выводы:
1. Адам работает лучше чем SGD.
2. батч-нормализация даёт значительное улучшение. Лучший результат у батч-норма на сверточных и полносвязных слоях.
3. Наулучший размер батча = 32.

## Свёрточная нейросеть здорового человека

**Задание 1.7 (5 баллов).** Наигравшись выше, обучим большую свёрточную сеть, которая даст на тестовой выборке __accuracy больше 80%__. В этом задании вам потребуется провести эксперименты, сравнив их между собой в конце. Возможно, будет несколько проще, если писать выводы во время или сразу после каждого эксперимента, после чего сделать общие выводы.

Рекомендуем начать с лучшей модели предыдущего задания и постепенно её улучшать. Вы можете использовать всё, что угодно: любые активации, сколь угодно большие свёрточные слои и глубокие сети. Единственное ограничение: __нельзя использовать предобученные сети и дополнительные данные__.

### Полезные советы
* Для начала неплохо бы научить что-нибудь побольше, чем 10 фильтров 3x3.
* __Главное правило: одно изменение на эксперимент__. Если у вас есть 2 идеи по улучшению сети, сначала попробуйте их независимо. Может оказаться, что одно из них дало __+10%__ точности, а другое __-7%__. А вы так и будете думать, что сделали 2 полезных изменения, которые в сумме дают __+3%__. Если какая-то идея не работает — даже если она вам нравится - опишите ее и выкидывайте из дальнейших экспериментов.
* __Be careful or you will dropout__. Дропаут (`keras.layers.Dropout`, `torch.nn.Dropout`) может позволить вам обучить в несколько раз бОльшую сеть без переобучения, выжав несколько процентов качества. Это круто, но не стоит сразу ставить dropout 50%. Во-первых, слишком сильный дропаут только ухудшит сеть (underfitting). Во-вторых, даже если дропаут улучшает качество, он замедляет обучение. Рекомендуем начинать с небольшого дропаута, быстро провести основные эксперименты, а потом жахнуть в 2 раза больше нейронов и дропаута ~~на ночь~~.
* __Аугментация данных__. Если котика слегка повернуть и подрезать (простите), он всё равно останется котиком. В Керасе есть [удобный класс](https://keras.io/preprocessing/image/), который поставит подрезание котиков на поток. Ещё можно сделать этот трюк в тесте: вертим картинку 10 раз, предсказываем вероятности и усредняем. Только один совет: прежде, чем учить, посмотрите глазами на аугментированные картинки. Если вы сами не можете их различить, то и сеть не сможет. [Аналогичный модуль для аугментации](https://pytorch.org/vision/stable/transforms.html) в Pytorch.
* __Don't just stack more layers__. Есть более эффективные способы организовать слои, чем простой Sequential. Вот пара идей: [Inception family](https://hacktildawn.com/2016/09/25/inception-modules-explained-and-implemented/), [ResNet family](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035?gi=9018057983ca), [Densely-connected convolutions](https://arxiv.org/abs/1608.06993). Только не копируйте архитектуру подчистую — вам скорее всего хватит меньшего размера.
* __Долго != плохо__. Более глубокие архитектуры обычно требуют бОльше эпох до сходимости. Это значит, что в первые несколько эпох они могут быть хуже менее глубоких аналогов. Дайте им время, запаситесь чаем и обмажьтесь batch-norm-ом.

In [ ]:
big_conv_model = nn.Sequential(
    nn.Conv2d(3, 128, 3),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),
     nn.Conv2d(128, 256, 3),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),
    nn.Conv2d(256, 256, 3),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.2),
    nn.Flatten(),
    nn.Linear(1024, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(),
    nn.Linear(1024, 10), 
    nn.Softmax(dim=1)
    )

In [ ]:
import torchvision
from torchvision import transforms as T 

aug_transforms = T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomVerticalFlip(p=0.5),
    T.ToTensor(),
])

test_aug_transforms = T.Compose([
    T.ToTensor()
])


dataset = torchvision.datasets.CIFAR10('cifar10/train/', download=True, transform=T.ToTensor(), train=True)
test_data = torchvision.datasets.CIFAR10('cifar10/test', download=True, transform=T.ToTensor(), train=False)
train_data, val_data = train_test_split(dataset, test_size=1000, random_state=42)

train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)
val_loader = torch.utils.data.DataLoader(val_data,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=2)

In [ ]:
model = CifarModel(deepcopy(big_conv_model), lambda model_params: Adam(model_params, lr=1e-3))
acc = train_model_experiment(model, 'task7_custom_model', train_loader, val_loader, test_loader, max_epoch=35)
acc

Момент истины: проверьте, какого качества достигла ваша сеть.

In [ ]:
from sklearn.metrics import accuracy_score
trainer = Trainer(gpus=1)
y_pred = torch.cat(trainer.predict(model, test_loader, return_predictions=True), dim=0).cpu()
print(np.array(y_pred))
test_acc = accuracy_score(y_test, y_pred)
print("\n Test_acc =", test_acc)
if test_acc > 0.8:
    print("Это победа!")

### График **accuracy** на валидационной выборке:
<img src="https://drive.google.com/uc?export=view&id=1HbmnEKi4br1wodROP-ng7x5qUWbdpHoT" width="600">

In [ ]:
del model

А теперь, опишите свои <s>ощущения</s> результаты от проведенных экспериментов. 

Увеличив количество сверточных слоев и пуллинга, а также добавив дропаут удалось побить требуемый порог качества.

# Часть 2. Fine-tuning обученных нейросетей

В этой части задания вам предстоит поработать с настоящими монстрами: сетями с почти сотней слоёв и десятками миллионов параметров. Например, такими:

![img](https://alexisbcook.github.io/assets/inception.png)
<center>googlenet inception v3</center>

Если внимательно всмотреться в картинку, можно заметить, что синим цветом обозначены свёрточные слои, красным — pooling, зелёным — конкатенация входов и т.п.

__Чем кормить такого монстра?__

Огромные нейросети обучаются на огромных массивах данных. В компьютерном зрении таких несколько, но самый популярный из них [ImageNet](http://image-net.org/). В этой выборке более миллиона изображений.

Задача этой сети состоит в классификации каждого изображения в один из 1000 классов. Вот они:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

classes = pickle.load(open('../input/ml2task1dataset/classes.pkl','rb'))
print(classes[::100])

## Зоопарк нейросетей

В пуполярных бибилотеках для содания нейросетей, кроме всего прочего, есть зоопарк предобученных нейросетей: [__`keras.applications`__](https://keras.io/applications/) для Keras и [torchvision.models](https://pytorch.org/vision/stable/models.html) для PyTorch. В этом задании мы предлагаем порадотать с моделью `InceptionV3`.

**Внимание!**
InceptionV3 требует много памяти для работы. Если ваш ПК начинает зависать:
* закройте всё кроме jupyter и браузера с одной вкладкой;
* если не помогло, загрузите эту тетрадку в [google colab](https://colab.research.google.com/) и работайте там;
* замените `InceptionV3` на `MobileNet`. Однако в этом случае вам придётся исправить и предобработку картинок.

Выберите оптимальный для вас вариант, загрузите модель (пример кода можно для каждого из фреймворков Keras и PyTorch можно посмотреть по ссылкам выше) и начнем работу!

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True, aux_logits=False).cuda()

Функция ниже позволяет найти для заданного изображения топ10 классов по мнению InceptionV3.

Для Keras:

In [ ]:
# import tensorflow as tf
# preprocess_input = tf.keras.applications.inception_v3.preprocess_input

Для PyTorch:

In [ ]:
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
preprocess_input = Compose([ToTensor(), Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [ ]:
from skimage.transform import resize


def predict_top10(img, model, preprocess_input):
    model.eval()
    img = resize(img, (299, 299), mode='reflect')
    assert img.min() >= 0.0 and img.max() <= 1.0
    plt.imshow(img)
    plt.show()
    img_preprocessed = preprocess_input(img * 255)[None]
    probs = torch.nn.functional.softmax(model(img_preprocessed.float().cuda()).detach().cpu()[0], dim=0)
    labels = reversed(probs.argsort()[-10:])

    print('top-10 classes:')
    for l in labels:
        print('%.4f\t%s' % (probs.ravel()[l], classes[l].split(',')[0]))

**Примечание:** не нужно бояться функции `tf.keras.applications.inception_v3.preprocess_input`: на самом деле это общее преобразование картинки (не привязанное непосредственно к `InceptionV3`), которое преобразует изображение в тензор и масштабирует фичи в отрехое $[-1; 1]$.  В PyTorch все преобрзаования над картинкой производятся при момощи модуля `torchvision.transform`, для преобразования в тензор используется класс `torchvision.transform.ToTensor`. Обратите внимание, что, в отличие от Temsorflow, родной входной формат для PyTorch - это значения в диапазоне $[0; 1]$.

`MobileNet`

Проверим, как она работает на близкой к обучающей выборке картинке:

In [ ]:
# predict_top10(plt.imread('albatross.jpg'))

А теперь попробуем ее на чем-то неожиданном!

In [ ]:
!wget http://cdn.com.do/wp-content/uploads/2017/02/Donal-Trum-Derogar.jpeg -O img.jpg
predict_top10(plt.imread('img.jpg'), model, preprocess_input)

## Dogs Vs Cats

А теперь попробуем построить классификатор, который отличает изображение кошки от собаки. 

![img](https://dingo.care2.com/pictures/greenliving/1203/1202163.large.jpg)

Скачайте данные из [Каггла](https://www.kaggle.com/c/dogs-vs-cats/data)

In [ ]:
# Этот ноутбук запускался на кагле, все данные подключались через апи датасетов.

## Sklearn way

**Задание 2.1 (1.5 балла).** В вашем распоряжении есть предобученная сеть InceptionV3. Ваша задача — обучить классификатор из sklearn (на ваш выбор), который будет отличать котов от собак, используя __активации нейронной сети в качестве признаков__.

Для начала прочитайте данные и сформируйте для вашего классификатора обучающую и тестовую выборки в пропорции 4:1. 

В вашем распоряжении всего 25 000 изображений различного размера, все в формате JPEG. Изображения кошек имеют название вида `./train/cat.*.jpg`, собак — `./train/dog.*.jpg`.

Считайте данные и для каждой картинки вычислите признаки из промежуточного слоя свёрточной сети. В качестве признаков можно выбрать какой-нибудь слой или несколько слоёв сети. Попробуйте найти комбинацию слоёв, которая работает лучше всего.

[Здесь](https://keras.io/getting-started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer) можно почитать, как посчитать активацию промежуточных слоёв.

In [ ]:
!rm -rf train
!unzip ../input/dogs-vs-cats/train.zip -d train;

In [ ]:
import os
import pandas as pd

filenames = os.listdir("./train/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
}) 
df.head(10)

Разделите данные на обучение и тест в отношении 4:1.

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train_filenames, X_test_filenames, y_train, y_test = train_test_split(df['filename'], df['category'], test_size=0.2, random_state=42)

Обучите поверх этих признаков классификатор из sklearn (можно попробовать несколько и выбрать лучший). Попробуйте получить ROC-AUC __хотя бы 99%__.

In [ ]:
# Убираем линейный слой и дропаут перед ним
model.dropout = nn.Identity()
model.fc = nn.Identity()

In [ ]:
import os
from skimage import io
from tqdm import tqdm
import gc
    
def get_features(root_dir, X_filenames, preprocess_input, feature_extractor):
    result = []
    for filename in tqdm(X_filenames):
        img_name = os.path.join(root_dir, filename)
        img = io.imread(img_name)
        img = resize(img, (299, 299), mode='reflect')
        assert img.min() >= 0.0 and img.max() <= 1.0
        img_preprocessed = preprocess_input(img * 255)[None].float()
        features = feature_extractor(img_preprocessed.cuda()).detach().cpu().numpy()
        result.append(features)
        del img_preprocessed, img
    gc.collect()
    return np.array(result)


In [ ]:
X_train_features = get_features('./train/train', X_train_filenames, preprocess_input, model)

In [ ]:
X_test_features = get_features('./train/train', X_test_filenames, preprocess_input, model)

In [ ]:
sklearn.__version__

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier


rdf =  RandomForestClassifier(n_estimators=500)
rdf.fit(X_train_features.squeeze(), y_train.to_numpy())

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test.to_numpy(), rdf.predict_proba(X_test_features.squeeze())[:, 1])

Опишите ваши выводы о проделанной работе.

## Fine-tuning

**Задание 2.2 (3 балла).** Давайте попробуем добиться ещё большего качества через дообучение (fine-tuning) модели. Новая цель — получить качество лучше, чем у классификатора из предыдущего пункта на признаках `InceptionV3`. Цель этого задания: получить значение ROC-AUC __не меньше 99.5%__.

__Шаг 1.__  Постройте сеть, в которой InceptionV3 "без головы" используется в качестве первого слоя. Поверх неё надстройте новую голову из `keras.layers`/ `torch.nn`— она будет отличать котов от собак. Это можно сделать с помощью [общего интерфейса модели](https://keras.io/models/model/). В PyTorch несеквенциальные модели можно строить, наследуя модель от класса `torch.nn.Module`: [общий интерфейс модели PyTorch](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

In [ ]:
class InceptionV3Model(nn.Module):
    def __init__(self, feature_extractor):
        super().__init__()
        self.feature_extractor = feature_extractor
        self.head = nn.Sequential(
          nn.Dropout(p=0.2),
          nn.Linear(2048, 1024),
          nn.ReLU(),
          nn.Linear(1024, 1)
        )

    def forward(self, x):
        features = self.feature_extractor(x)
        return self.head(features)

__Шаг 2.__ Обучите "голову" на обучающей выборке, не меняя весов изначальной сети. Это называется обучением с замороженными весами. Как это сделать в Keras, можно прочитать [здесь](https://keras.io/getting-started/faq/#how-can-i-freeze-keras-layers), для PyTorch смотрите [статью](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html), подраздел про Finetuning. В PyTorch заморозка весов регулируется параметром `requires_grad = False`.

In [ ]:
from torch.utils.data import Dataset

class CatVSDogDataset(Dataset):
    def __init__(self, filenames, labels, root_dir, preprocess_input, train, transform=None):
        self.root_dir = root_dir
        self.filenames = filenames
        self.labels = labels
        self.transform = transform
        self.preprocess_input = preprocess_input
        self.train = train
    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.filenames.iloc[idx])
        image = io.imread(img_name)
        image = resize(image, (299, 299), mode='reflect')
        assert image.min() >= 0.0 and image.max() <= 1.0
        image_preprocessed = preprocess_input(image * 255).float()
        label = self.labels.iloc[idx]

        if self.transform:
            image_preprocessed = self.transform(image_preprocessed)
        if self.train:
            return (image_preprocessed, label)
        else: 
            return image_preprocessed

In [ ]:
train_dataset = CatVSDogDataset(X_train_filenames, y_train, './train/train', preprocess_input, train=True)
test_dataset = CatVSDogDataset(X_test_filenames, y_test, './train/train', preprocess_input, train=False)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=64,
                                          shuffle=False,
                                          num_workers=2)

In [ ]:
class CatVsDogModel(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.model.forward(x)
      
    def configure_optimizers(self):
        return Adam(filter(lambda p: p.requires_grad, self.model.parameters()), lr=2e-3)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x).squeeze()
        loss = self.loss(logits, y.float())
        
        preds = (logits > 0).float()
        acc = accuracy_score(y.cpu(), preds.cpu())
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
      
    def predict_step(self, batch, batch_idx: int, dataloader_idx: int = None):
        return (self(batch) > 0).float()

In [ ]:
feature_extractor = deepcopy(model)
for param in feature_extractor.parameters():
    param.requires_grad = False

In [ ]:
logger = TensorBoardLogger("logs/part2_freeze")
net = InceptionV3Model(feature_extractor)
cvd_model = CatVsDogModel(net)
trainer = Trainer(logger=logger, gpus=1, max_epochs=2)
trainer.fit(cvd_model, train_loader)

In [ ]:
from sklearn.metrics import roc_auc_score
y_pred = torch.cat(trainer.predict(cvd_model, test_loader, return_predictions=True), dim=0).cpu()
roc_auc_score(y_test, y_pred)

__Sanity check:__ После этого шага ваша модель должна уже быть сравнима по точности с моделями из задания 1.

Если всё получилось, самое время [сохранить модель Keras](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model) или [сохранить модель PyTorch](https://pytorch.org/tutorials/beginner/saving_loading_models.html).

In [ ]:
trainer.save_checkpoint("inceptionv3_model.ckpt")

__Шаг 3.__ "Разморозьте" несколько предыдущих слоёв модели и продолжите обучение. На этом этапе важно не переобучиться: смотрите качество на валидации.

Если качество не улучшается, а сразу идёт вниз, попробуйте уменьшить число обучаемых слоёв или воспользуйтесь [аугментацией данных](https://keras.io/preprocessing/image/) ([аугментация](https://pytorch.org/vision/stable/transforms.html) в Pytorch). В общем случае всегда полезно помнить про аугментацию данных, даже если и без неё всё работает — иногда она творит [чудеса](https://medium.com/nanonets/how-to-use-deep-learning-when-you-have-limited-data-part-2-data-augmentation-c26971dc8ced).

In [ ]:
feature_extractor = deepcopy(model)
for param in list(feature_extractor.parameters())[:-3]:
    param.requires_grad = False

for param in list(feature_extractor.parameters())[-3:]:
    param.requires_grad = True

In [ ]:
logger = TensorBoardLogger("logs/task2_unfreeze")
net = InceptionV3Model(feature_extractor)
cvd_model = CatVsDogModel(net)
trainer = Trainer(logger=logger, gpus=1, max_epochs=2)
trainer.fit(cvd_model, train_loader)

__Шаг 4.__ Вычислите финальное качество.

In [ ]:
y_pred = torch.cat(trainer.predict(cvd_model, test_loader, return_predictions=True), dim=0).cpu()
roc_auc_score(y_test, y_pred)

Напишите отчёт и вознаградите себя за старания чем-нибудь.

### Вывод:
RandomForest, обученный на фичах получает **accuracy**~0.75. В то время как полносвязные слои с замороженными свертками получают **accuracy**~0.975. Размораживание 2 последних сверток не даёт улучшений. Возможно нужно разморозить больше слоёв.

# Всё сделали, но азарт не прошел?

В таких случаях можно пробовать следующие техники:
* Ансамбль из нескольких предобученных нейросетей. Bagging? Stacking? Boosting? Всё, что пожелаете.
* Более честный эксперимент: разделяем данные на train/__dev__/test, все сравнения делаем по dev, а test используем только в самом конце.
* Аугментировать данные картинками из интернета. Уж чего, а котиков и собачек там хватает.

# Часть 3. Рекуррентные языковые модели

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSj85jp-W-V-Bz8ZBjFJYIkV1TTxQxTMh4iqls_rRt8O-sraL08PA)

В этой части домашней работы мы создадим языковую модель на рекуррентных нейросетях (RNN) и заставим её придумывать имена.

__Языковая модель__, если вкратце, — это модель, которая умеет предсказывать вероятность некоторого текста. Ее можно использовать также для генерирации нового текста в соответствии с обученными вероятностями. Задание будет заключаться в том, чтобы научить модель генерировать новые имена, скормив ей для этого 8к существующих.

В данном случае в качестве входных данных мы будет работать со строками, которые можно рассматривать как последовательности _символов_: $\{x_0, x_1, x_2, ..., x_n\}$. 

Наша основная задача — научиться предсказывать вероятность следующего символа:
$$ p(x_0, x_1, x_2, ..., x_n) = \prod_t p(x_t | x_0, ... x_{t - 1}) $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Данные

Мы будем строить языковую модель по ~8k человеческих имён на латинице. Если когда-нибудь вам нужно будет дать имя своему ребёнку, у вас будет для этого генеративная нейросетевая модель.

Давайте их прочитаем:
* Считайте все строки из файла `names` в список
* В начало каждой строки допишите __пробел__
* В конце сроки не должно быть переноса (`\n`)

In [ ]:
import os
start_token = " "

# YOUR CODE
with open('../input/names-ml2task1/names') as file:
    lines = file.readlines()
    lines = [' ' + line[:-1] for line in lines]

In [ ]:
assert all(line[0] == start_token for line in lines)
assert all(line[-1] != '\n' for line in lines)

In [ ]:
print ('n samples = ',len(lines))
for x in lines[::1000]:
    print(x)

Проверьте, что все корректно:

In [ ]:
MAX_LENGTH = max(map(len, lines))
print("max length =", MAX_LENGTH)
assert MAX_LENGTH == 16 , "max length (for names) should be 16. remove assert if you work on different dataset"

## Словари

В начале нам будет необходимо построить "словарь" — упорядоченное множество уникальных символов, которые сеть может породить. Это нужно, чтобы уметь сопоставить каждому символу свой номер. Перед отправкой в сеть все символы будут кодироваться их номерами в словаре.

Также необходимо добавить в словарь пробельный символ, который будет использоваться в качестве специального токена.

In [ ]:
import string
tokens = string.ascii_letters + ' ' + '-' + '\''

tokens = sorted(list(tokens))

n_tokens = len(tokens)
print ('n_tokens = ', n_tokens)

assert 50 < n_tokens < 60

А теперь построим обратный словарь: для каждой буквы посчитаем её номер в списке токенов.

In [ ]:
token_to_id = { tokens[idx]:idx for idx in range(len(tokens))}

И проверим, все ли корректно:

In [ ]:
assert len(tokens) == len(token_to_id), "число токенов должно совпадать"

for i in range(n_tokens):
    assert token_to_id[tokens[i]] == i, "словарь должен указывать на индекс буквы в tokens"

print("Кажется заработало...")

Имея построенное соответствие, можно преобразовать батч входных данных в матрицу int32 номеров токенов. Так как в батче все строки должны быть одной длины, слишком короткие строки в батче нужно будет дополнить пробелами (паддинг).

In [ ]:
def to_matrix(lines, max_len=None, pad=token_to_id[' '], dtype='int32'):
    """Casts a list of names into rnn-digestable matrix"""
    max_len = max_len or max(map(len, lines))
    lines_ix = np.zeros([len(lines), max_len], dtype) + pad

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.get, lines[i]))
        lines_ix[i, :len(line_ix)] = line_ix

    return lines_ix

In [ ]:
print('\n'.join(lines[::2000]))
print(to_matrix(lines[::2000]))

## Один шаг RNN

Рекуррентная нейронная сеть (RNN) — это такая сеть с <s>блокнотом</s> состоянием $h$, в который она умеет писать то, что видела.

Сеть начинает с пустого $h_0 = \vec 0$, после чего текст обрабатывается по одному символу:
* $x_t$ — очередной символ, $h_t$ — предыдущее состояние
* $h_{t+1} = \text{get_h_next}(h_t, x_t)$ — новое состояние
* $p(x_{t+1} | h_{t+1}) = \text{get_probs}(h_{t+1})$ — вероятность следующего символа



<img src="https://i.imgur.com/8l4qFF0.png" width=480>

Поскольку $x_t$ — это индекс символа в словаре (натуральное число), то ему можно сопоставить некоторый обучаемый вектор (*embedding*).

**Задание 3.1 (1 балл)**. Реализуйте вычисление нового состояния *get_h_next* и вероятности следующего символа *get_probs*, после чего напишите код для одного шага рекуррентной сети *rnn_one_step*, как на схеме выше.

In [ ]:
import tensorflow.compat.v1 as tf
import keras, keras.layers as L # torch.nn as L
tf.disable_v2_behavior()  

emb_size, rnn_size = 16, 64

Создадим слой, который сопоставляет каждому из n_tokens входов свой обучаемый вектор:

In [ ]:
embed_x = L.Embedding(n_tokens, emb_size)

Теперь инициализируем слой, вычисляющий следующее состояния $[emb(x_t), h_t] \to h_{t+1}$.

In [ ]:
get_h_next = L.Dense(rnn_size, activation="tanh", name="layer1_rnn")

И, наконец, слой предсказывающий вероятности $h_{t+1} \to P(x_{t+1}|h_{t+1})$.

In [ ]:
get_probs = L.Dense(n_tokens, activation="softmax", name="layer2_rnn")

Для реализации одного шага RNN реализуйте следующую последовательность действий:
1. замените номер символа на его вектор (embedding) (*hint*: возможно, вам потребуется tf.reshape);
2. сконкатенируйте вектор входа и предыдущее состояние;
3. вычислите следующее состояние сети;
4. предскажите вероятности для языковой модели P(x_next | h_next).

In [ ]:
def rnn_one_step(x_t, h_t):    
    # YOUR CODE
    embedding = embed_x(tf.reshape(x_t, (-1,1)))[:,0,:] 
    h_t_reshaped = tf.reshape(h_t, (-1, rnn_size))
    hidden_x = tf.concat([embedding, h_t_reshaped], 1)
    hidden_x_reshaped = tf.reshape(hidden_x, (-1, emb_size + rnn_size))
    h_next = get_h_next(hidden_x_reshaped)
    output_probs = get_probs(h_next)
    return h_next, output_probs

Проверим, что все работает (для PyTorch проверочный код разрешается изменить):

In [ ]:
input_sequence = tf.placeholder('int32', (None, MAX_LENGTH))
batch_size = tf.shape(input_sequence)[0]

# начальное состояние из нулей
h0 = tf.zeros([batch_size, rnn_size])

In [ ]:
h1, p_y1 = rnn_one_step(input_sequence[:, 0], h0)

dummy_data = np.arange(MAX_LENGTH * 2).reshape([2, -1])
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
test_h1, test_p_y1 = sess.run([h1, p_y1],  {input_sequence: dummy_data})
assert test_h1.shape == (len(dummy_data), rnn_size)
assert test_p_y1.shape == (len(dummy_data), n_tokens) and np.allclose(test_p_y1.sum(-1), 1)

## Много шагов RNN

После того как был реализован один шаг нейросети, самое время сделать этих шагов побольше. Самый простой способ это сделать — написать цикл для фиксированного числа шагов (`MAX_LENGTH`).

**Задание 3.2 (1 балл)**. Реализуйте много шагов рекуррентной сети, на каждом шаге вычисляя следующее состояние RNN, исходя из предыдущего, при этом не забывая про *get_h_next* и *get_probs*.

In [ ]:
h_prev = h0
predicted_probs = []

for t in range(MAX_LENGTH):
    x_t = input_sequence[:, t]
    # YOUR CODE
    h_next, probs_next = rnn_one_step(x_t, h_prev)
    
    # END OF YOUR CODE
    predicted_probs.append(probs_next)
    h_prev = h_next
    
predicted_probs = tf.stack(predicted_probs, axis=1) # torch.stack for PyTorch

In [ ]:
assert predicted_probs.shape.as_list() == [None, MAX_LENGTH, n_tokens]
assert h_prev.shape.as_list() == h0.shape.as_list()

## Обучение RNN

Как и любую вероятностную модель, RNN можно обучить методом максимизации log-правдоподобия по всей выборке $D$:

$$ \theta = \underset \theta {argmax} \log P(D) $$

где
$$ \log P(D) = \underset {\vec x \in D} \sum \log P(\vec x) = \underset {\vec x \in D} \sum \underset {x_t \in \vec x} \sum \log P(x_t | x_0, ..., x_{t+1})$$

C тем же успехом мы можем __минимизировать__ кроссэнтропию — то же самое, но с минусом.

In [ ]:
predictions_matrix = predicted_probs[:, :-1]
answers_matrix = tf.one_hot(input_sequence[:, 1:], n_tokens)  # torch.nn.functional.one_hot for PyTorch

print('predictions_matrix:', predictions_matrix.shape)
print('answers_matrix:', predictions_matrix.shape)

**Задание 3.3 (2 балла)**. Реализуйте вычисление функции потерь (кроссэнтропия) и шаг градиентного спуска.

In [ ]:
import tensorflow
loss = tensorflow.reduce_mean(tensorflow.losses.categorical_crossentropy(answers_matrix, predictions_matrix))
optimize = tf.train.AdamOptimizer().minimize(loss)

### Цикл обучения

**Задание 3.4 (1 балл)**. Напишите цикл обучения:
1. выбираем `batch_size` случайных строчек
2. преобразуем их в матрицу индексов
3. вычисляем функцию потерь и делаем шаг обучения
4. записываем функцию потерь в `history`

Для удобства отладки рекомендуем печатать или рисовать промежуточные результаты раз в несколько итераций.

In [ ]:
batch_size = 32
history = []

sess.run(tf.global_variables_initializer()) # эту строку можно выпилить

In [ ]:
words_matrix = to_matrix(lines, max_len=16)

In [ ]:
for i in range(1000):
    batch = words_matrix[np.random.choice(words_matrix.shape[0], batch_size, replace=False), :]
    _loss, _ = sess.run((loss, optimize), {input_sequence: batch})
    history.append(_loss)

In [ ]:
plt.plot(history)
plt.grid()

## Применение RNN

Только что у нас обучилась модель, которая предсказывает вероятности следующего символа.
Теперь давайте применим её к строке из одного пробела. Получим вероятности первой буквы имени. После чего:
* $x_t \sim P(x_t | h_t)$ — выберем букву пропорционально вероятностям.
* $h_{t+1} = \text{get_h_next}(h_t, x_t)$ — присоединим букву к имени и прогоним через RNN

Для начала инициализируем необходимые переменные:

In [ ]:
x_t = tf.placeholder('int32', (None, ))
h_t = tf.Variable(np.zeros([1, rnn_size], 'float32'))

next_h, next_probs = rnn_one_step(x_t, h_t)

**Задание 3.5 (1 балл).** Напишите функцию, генерируюущю новые имена:

In [ ]:
def generate_sample(seed_phrase=' ', max_length=MAX_LENGTH):
    result = seed_phrase
    words = to_matrix(seed_phrase)

    sess.run(tf.assign(h_t, h_t.initial_value))

    for word in words[:-1]:
        sess.run(tf.assign(h_t, next_h), {x_t: word})

    word = words[-1][0]
    for _ in range(len(seed_phrase), max_length):
        probs, _ = sess.run([next_probs, tf.assign(h_t, next_h)], {x_t: [word]})
        word = np.random.choice(n_tokens, p=probs[0])
        result += tokens[word]
    return result

Посмотрим, что же придумала наша модель:

In [ ]:
for _ in range(10):
    print(generate_sample())

In [ ]:
for _ in range(25):
    print(generate_sample(' Putin'))

### Что теперь?

Если вам наскучит решать повседневные задачи или вам нужны новые идеи, вы теперь всегда можете воспользоваться RNN, чтобы сгенерировать что-то новое. Вот несколько задач, от которых можно отталкиваться:
* названия статей по глубинному обучению;
* названия карт Magic The Gathering;
* [имена покемонов](https://github.com/cervoise/pentest-scripts/blob/master/password-cracking/wordlists/pokemon-list-en.txt);
* clickbait заголовки;
* молекулы в формате [smiles](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system);
* ваша фантазия, с ограничениями которой вы уже должны были понять, как бороться.

Если возьмётесь за эту задачу, то вот несколько полезных советов:
* Сейчас модель обучается на коротких строчках. Если у вас роман, его придётся порезать на кускочки.
* Если длина строк сильно варьируется, можно поставить параметр MAX_LENGTH так, чтобы он покрывал 90%. Это обычно дает ускорение примерно в 2 раза.
* Для более сложных задач требуется больше нейронов (rnn_size). Кроме того, можно экспериментировать и со составляющими сети (см. ниже).

### Ещё почитать

* [Подборка советов](https://danijar.com/tips-for-training-recurrent-neural-networks/) по обучению RNN. Чуть более полезная, чем обычно.
* Отличный блог-пост от Andrej Karpathy про языковые модели на rnn, их применение и визуализацию — [Unreasonable Effectiveness of RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).
* Большой список статей, постов, реализаций и прочих полезностей по RNN - [awesome rnn](https://github.com/kjw0612/awesome-rnn).
* Зоопарк готовых рекуррентных ячеек (LSTM, GRU) в [Керасе](https://keras.io/layers/recurrent/) и [PyTorch](https://pytorch.org/docs/stable/nn.html#recurrent-layers).
* Сейчас мы настраиваем количество итераций заранее. Если вы хотите определять их динамически, милости просим в [tf.while_loop](https://www.tensorflow.org/api_docs/python/tf/while_loop) или [tf.scan](https://www.tensorflow.org/api_docs/python/tf/scan).
* А ещё рекуррентные сети можно аугментировать механизмом внимания или долговременной памятью. Вот тут есть [хорошая статья](https://distill.pub/2016/augmented-rnns/).